In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelBinarizer

# File system manangement
import os

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import MinMaxScaler, Imputer, LabelBinarizer
from sklearn.linear_model import LogisticRegression
import sys
sys.path.insert(0, '../scripts/')

#Import custom transformers
from data_loader import extract_X_y, splitting, load_data
from pipeline_utils import ToDummiesTransformer, TypeSelector,DataFrameFeatureUnion,DropColumnsTransformer, \
ToStandardScaler,DataFrameFunctionTransformer, DropAllZeroTrainColumnsTransformer, MultiColumnLabelEncoder, CustomLabelBinarizer, LabelBinarizerPipelineFriendly, \
DummyEncoder

from catboost import CatBoostClassifier, FeaturesData, Pool

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


/home/sejal/anaconda2/envs/home_credit/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sejal/anaconda2/envs/home_credit/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sejal/anaconda2/envs/home_credit/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sejal/anaconda2/envs/home_credit/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data_path = '../data/train.csv'
data_path_pkl = '../data/train.pkl'

fulldata = load_data(data_path, data_path_pkl)
X, y = extract_X_y(fulldata)
X_train, X_val, y_train, y_val = splitting(X, y, val_split=0.2, random_state=42)

INFO:data_loader:
Timings: 'load_data'  3.08 ms
INFO:data_loader:
Timings: 'extract_X_y'  1.98 ms
INFO:data_loader:
Timings: 'splitting'  59.71 ms


### Make Pipeline

In [3]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


In [4]:
cat_cols = X_train.select_dtypes(include=["object"]).columns
object_levels = np.union1d(X_train[cat_cols].fillna('NAN'), X_val[cat_cols].fillna('NAN'))

In [5]:
numerical = make_pipeline(
    TypeSelector("number"),
    DataFrameFunctionTransformer(func = lambda x: x.astype(np.float64)),
    DataFrameFunctionTransformer(func = np.mean, impute=True),
    ToStandardScaler()
)

categorical = make_pipeline(
    TypeSelector('object'),
    #DataFrameFunctionTransformer(lambda x:x.astype('category', categories=object_levels)),
    #ToDummiesTransformer(),
    #DropAllZeroTrainColumnsTransformer()
)

In [7]:
preprocessing_features = DataFrameFeatureUnion([numerical, categorical])

In [8]:
classifier_pipeline = make_pipeline(
    DropColumnsTransformer(['PassengerId']),
    preprocessing_features,
    LogisticRegression(random_state=42)
)

In [ ]:
classifier_pipeline.fit(X_train, y_train)
y_pred = classifier_pipeline.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

In [9]:
preprocess_df = preprocessing_features.fit_transform(X_train)
preprocess_df_test = preprocessing_features.fit_transform(X_val)

In [79]:
num_feature_index = [preprocess_df.columns.get_loc(col) for col in preprocess_df.select_dtypes(include=np.number).columns]
cat_feature_index = [preprocess_df.columns.get_loc(col) for col in preprocess_df.select_dtypes(include=np.object).columns]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [83]:
train_data = Pool(
    data=FeaturesData(
        num_feature_data=np.array(num_feature_index, dtype=np.float32),
        cat_feature_data=np.array(cat_feature_index, dtype=np.float32)
    ),
    label=list(preprocess_df.columns)
)
test_data = Pool(
    data=FeaturesData(
        num_feature_data=np.array(preprocess_df_test.select_dtypes(include=np.number).columns, dtype=np.float32),
        cat_feature_data=np.array(preprocess_df_test.select_dtypes(include=np.object).columns, dtype=np.object)
    ),
    label=list(preprocess_df_test.columns)
)

CatboostError: num_feature_data must be 2D numpy.ndarray, it has shape (6,) instead